In [1]:
import dlt
import requests
import pandas as pd 
import json 
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.paginators import OffsetPaginator
from dlt.sources.helpers.rest_client.paginators import PageNumberPaginator
from dlt.sources.helpers.rest_client.auth import APIKeyAuth
from dlt.destinations import filesystem

In [2]:
# Define the API endpoint and your access key
API_URL = 'https://api.marketstack.com/v1/eod'
ACCESS_KEY = 'hoho'


In [3]:
# Extract raw data, auto normalization and Load data into GCP Bucket 
@dlt.resource(name="stock", write_disposition="replace")
def extract_raw_full_refresh(date_from, date_to):
    client = RESTClient(
        base_url=API_URL,
        paginator=OffsetPaginator(  # Handles pagination using offset
            limit=100, offset=0, total_path=None
        )
    )

    for page in client.paginate(
        params={
            'access_key': ACCESS_KEY,
            'symbols': 'AAPL,MSFT',  # Example symbols
            'date_from': date_from,
            'date_to': date_to
        }
    ):
        yield page

In [ ]:
@dlt.resource(name="stock", write_disposition="append")
def extract_raw_incremental(cursor_date=dlt.sources.incremental(
        "date",   # <--- field to track, our timestamp
        initial_value=initial_dt  
        )
    ):
    client = RESTClient(
        base_url=API_URL,

        paginator=OffsetPaginator(  # Handles pagination using offset
            limit=100, offset=0, total_path=None
        )
    
    )

    for page in client.paginate(
        params = {
            'access_key': ACCESS_KEY,
            'symbols': 'AAPL,MSFT',  # Example symbols
            'date_from': cursor_date.last_value,
            'date_to': datetime.now().strftime("%Y-%m-%d")
        }
    ):
        yield page 


In [4]:
GCP_URL = "gs://de-zoomcamp-project-453801-terra-bucket" 
gcp_bucket = filesystem(GCP_URL)

In [12]:
# Load credentials from JSON file
with open("/home/chenchen/.gc/my-creds.json") as f:
    credentials = json.load(f)

# Initialize filesystem with credentials
gcp_bucket = filesystem(GCP_URL, credentials=credentials)

In [13]:
pipeline = dlt.pipeline(
    pipeline_name="marketstack_pipeline",
    destination=gcp_bucket,  # Set the destination to GCS
    dataset_name='stock_dataset'
)

In [16]:
# run the pipeline with the new resource
load_info = pipeline.run(extract_raw, write_disposition="replace",loader_file_format="csv" )

print(load_info)

Pipeline marketstack_pipeline load step completed in 1.85 seconds
1 load package(s) were loaded to destination filesystem and into dataset stock_dataset
The filesystem destination used gs://de-zoomcamp-project-453801-terra-bucket location to store data
Load package 1742107479.6691566 is LOADED and contains no failed jobs
